In [1]:
import fastai
import fire
import numpy as np

from fastai import *
from fastai.text import *
import torch
from fastai_contrib.utils import read_file, read_whitespace_file,\
    DataStump, validate, PAD, UNK, get_sentencepiece

import pickle

from pathlib import Path

from collections import Counter

In [13]:
dir_path = 'data/wiki/ja-2/'
spm_dir = 'data/wiki/ja/'
lang='ja'
cuda_id=0
qrnn=True
subword=True
max_vocab=16000
bs=70
bptt=70
name='wt-2'
num_epochs=1
ds_pct=1.0
name = 'wt-100'

In [3]:
results = {}
model_dir = 'models' # removed from params, as it is absolute models location in train_clas and here it is relative
if not torch.cuda.is_available():
    print('CUDA not available. Setting device=-1.')
    cuda_id = -1
torch.cuda.set_device(cuda_id)

dir_path = Path(dir_path)
assert dir_path.exists()
model_dir = Path(model_dir)
model_dir.mkdir(exist_ok=True)

if spm_dir:
    spm_dir=Path(spm_dir)
else:
    spm_dir = dir_path
assert spm_dir.exists()

print('Batch size:', bs)
print('Max vocab:', max_vocab)
model_name = 'qrnn' if qrnn else 'lstm'
if qrnn:
    print('Using QRNNs...')

trn_path = dir_path / f'{lang}.wiki.train.tokens'
val_path = dir_path / f'{lang}.wiki.valid.tokens'
tst_path = dir_path / f'{lang}.wiki.test.tokens'
for path_ in [trn_path, val_path, tst_path]:
    assert path_.exists(), f'Error: {path_} does not exist.'

if subword:
    # apply sentencepiece tokenization
    trn_path = dir_path / f'{lang}.wiki.train.tokens'
    val_path = dir_path / f'{lang}.wiki.valid.tokens'

    read_file(trn_path, 'train')
    read_file(val_path, 'valid')
#     read the already whitespace separated data without any preprocessing
#     trn_tok = read_whitespace_file(trn_path)
#     val_tok = read_whitespace_file(val_path)

    # assume sentencepiece training is done after merge of wiki
    # here we're just loading the trained spm model
    sp = get_sentencepiece(spm_dir, None, 'wt-all', vocab_size=max_vocab)
    kwargs = dict(mark_fields=False)
    data_lm = TextLMDataBunch.from_csv(dir_path, 'train.csv',
                                        **sp)
#     data_lm = TextLMDataBunch.from_tokens(dir_path, trn_tok, None, val_tok, None, sp['vocab'])
    itos = data_lm.train_ds.vocab.itos
    stoi = data_lm.train_ds.vocab.stoi

In [7]:
vocab=sp['vocab']


In [12]:
assert vocab.itos == itos

In [10]:
assert stoi == vocab.stoi

In [15]:
pickle.dump(itos,open(Path('data/wiki/ja-100')/ Path('models') / f'itos_{name}.pkl','wb'))

In [17]:
Path('data/wiki/ja-100')/ Path('models')/ f'itos_{name}.pkl'

PosixPath('data/wiki/ja-100/models/itos_wt-100.pkl')

In [11]:
sp

{'tokenizer': Tokenizer SentencepieceTokenizer in data/wiki/ja/models with the following rules:
  - fix_html
  - replace_rep
  - replace_wrep
  - deal_caps
  - spec_add_spaces
  - rm_useless_spaces,
 'vocab': <fastai.text.transform.Vocab at 0x7f77528da9e8>}

In [12]:
data_lm

TextLMDataBunch;
Train: LabelList
y: LMLabel (69293 items)
[0 0 0 0 ... 0 0 0 0]
Path: .
x: TextList (69293 items)
[list([5, 1186, 1186, 725, 865, 944, 5, 24])
 list([5, 1186, 1186, 725, 865, 944, 5, 24, 7229, 1353, 12, 53, 19, 2, 1981, 730, 9, 2927, 2860, 15, 58, 23, 275, 2172, 4825, 283, 7416, 2443, 730, 7, 1849, 21, 4, 4715, 3, 13868, 46, 2172, 4825, 10, 2, 9721, 1021, 232, 1304, 7, 6916, 21, 4, 160, 3636, 12, 2, 1021, 232, 1304, 7, 495, 301, 1294, 125, 760, 966, 8, 7815, 25, 12295, 21, 4, 163, 5987, 263, 2, 213, 3, 10767, 111, 217, 973, 1813, 301, 2172, 4825, 1283, 534, 3, 259, 1427, 9050, 7, 12295, 15767, 2, 1025, 1523, 0, 310, 4, 11073, 34, 3090, 213, 3, 4529, 109, 6500, 2, 8415, 60, 47, 2172, 4825, 6, 4715, 1908, 6152, 5079, 21, 4])
 list([5, 1186, 1186, 725, 865, 944, 5, 24])
 list([5, 1186, 1186, 725, 865, 944, 5, 24, 5053, 409, 12, 2, 2172, 4825, 283, 325, 11772, 9, 78, 9, 973, 1813, 6, 9023, 9038, 204, 7145, 397, 8, 2473, 152, 4, 2095, 448, 101, 3666, 9, 884, 13147, 26, 4])


In [13]:
itos

['<unk>',
 '<pad>',
 '、',
 'の',
 '。',
 '▁',
 'は',
 'が',
 'を',
 '・',
 'に',
 ')',
 '年',
 '(',
 'で',
 'と',
 '.',
 '」',
 '「',
 '月',
 '▁"',
 'した',
 'から',
 '2',
 '1',
 'し',
 'である',
 'や',
 'も',
 '日',
 '3',
 'する',
 'として',
 'た',
 '4',
 'では',
 '』',
 '『',
 '5',
 'ス',
 'には',
 'された',
 'る',
 '6',
 '大',
 'この',
 'な',
 '年に',
 '人',
 '7',
 'など',
 'また',
 'している',
 '10',
 'という',
 'その',
 'して',
 '8',
 '第',
 'て',
 '9',
 '▁-',
 'ア',
 'ト',
 '-',
 '市',
 '""',
 '12',
 '山',
 '一',
 'ク',
 'ている',
 'り',
 '本',
 '日に',
 'され',
 'い',
 '中',
 'ド',
 'ラ',
 '日本',
 '11',
 'となった',
 'による',
 'ズ',
 'か',
 'によって',
 ':',
 '国',
 '川',
 ',',
 '町',
 'により',
 'であり',
 'であった',
 'される',
 '▁(',
 '上',
 'マ',
 '回',
 'カ',
 'リ',
 'く',
 '後',
 '長',
 'されている',
 '名',
 '子',
 '新',
 'へ',
 '田',
 '部',
 '同',
 '法',
 '18',
 'ナ',
 'がある',
 '駅',
 '者',
 'となる',
 'より',
 '線',
 'タ',
 '家',
 '水',
 'ル',
 'ため',
 '地',
 '、「',
 'ロ',
 'への',
 '島',
 'まで',
 'き',
 'つ',
 '分',
 'バ',
 '戦',
 '化',
 'イ',
 's',
 '内',
 'み',
 '三',
 'シ',
 '道',
 '数',
 'コ',
 '的な',
 'サ',
 'ら',
 '20',
 'った',
 'していた

In [14]:
tokenizer=sp['tokenizer']

In [15]:
data_lm.valid_ds

LabelList
y: LMLabel (17324 items)
[0 0 0 0 ... 0 0 0 0]
Path: .
x: TextList (17324 items)
[list([5, 1186, 1186, 725, 865, 944, 5, 24])
 list([5, 1186, 1186, 725, 865, 944, 5, 24, 304, 218, 856, 6640, 36, 6, 2884, 12, 34, 19, 34, 673, 285, 53, 19, 24, 1192, 2, 5, 1186, 1186, 7759, 5, 479, 660, 836, 5410, 37, 1225, 543, 1107, 36, 58, 5644, 4615, 262, 4, 37, 1225, 543, 1107, 36, 7, 37, 1914, 15, 1351, 36, 8043, 22, 6691, 59, 465, 5434, 119, 3, 8, 1700, 31, 11546, 1691, 4233, 4])
 list([5, 1186, 1186, 725, 865, 944, 5, 24])
 list([5, 1186, 1186, 725, 865, 944, 5, 24, 304, 1225, 543, 1107, 36, 35, 11436, 128, 7857, 1199, 107, 4, 2197, 3, 1458, 1199, 107, 1190, 3319, 2719, 54, 314, 7414, 4747, 7, 12589, 13, 314, 5933, 14, 10662, 5856, 11, 4])
 ...
 list([5, 1186, 1186, 725, 865, 944, 5, 24, 5, 6457, 2992, 1607, 6, 2, 6843, 10, 129, 9295, 89, 22, 91, 8, 814, 617, 4, 5875, 3, 5633, 6, 9395, 2359, 675, 5868, 4, 67, 478, 3566, 2006, 949, 3, 2624, 6, 2, 13611, 4330, 359, 9, 78, 9, 100, 5409, 102

In [16]:
with open(val_path, 'r') as f:
    for i,t in enumerate(f):
        print(i)
        print(t)
        if i > 10:
            break

0
秋元与助

1


2
秋元 与助（あきもと よすけ）は、幕末の下野喜連川藩の家老。

3


4
喜連川生まれで、秋元家は曽祖父の広伴の時代から喜連川藩に侍医・儒臣として仕えた家柄で、与助の代で4代目となる。喜連川藩の第9代藩主である煕氏に仕えたが、天保6年（1835年）に侍医を辞めて儒臣として藩に仕えている。煕氏の藩政改革に協力し、藩校の翰林館が設立されると、その教頭となって子弟の教育に務めた。

5


6
煕氏の死後、家老の二階堂貞明が失脚すると家老として藩政をとる。倹約や貧民救済、文学振興に務め、明治元年（1868年）には藩の権大参事となった。明治2年（1869年）に明治政府によって公議所が設立されると、公議人に任命され、公議所が集議院と改称されると集議員に任命された。この年の9月27日には文化人であることが広く知られていたため、明治天皇の御前で講演を務めている。版籍奉還後は全ての官位を辞して隠退し、以後は史書編纂などの文化活動に専念した。その関係で明治16年（1883年）に栃木県史誌編輯史料取調委員に任命された。

7


8
明治18年（1885年）3月16日に死去した。享年85。

9


10
晩年の著書は千以上になり、主な著作に『栃木県史』や『塩谷紀略』などがある。なお、与助は明治になって梅園と号したが、これは彼が梅が大好きで藩校に何本も梅を植えたことなどに由来するという。

11




In [17]:
tmp = []
for id in [5, 1186, 1186, 725, 865, 944, 5, 24, 304, 218, 856, 6640, 36, 6, 2884, 12, 34, 19, 34, 673, 285, 53, 19, 24, 1192, 2, 5, 1186, 1186, 7759, 5, 479, 660, 836, 5410, 37, 1225, 543, 1107, 36, 58, 5644, 4615, 262, 4, 37, 1225, 543, 1107, 36, 7, 37, 1914, 15, 1351, 36, 8043, 22, 6691, 59, 465, 5434, 119, 3, 8, 1700, 31, 11546, 1691, 4233, 4]:
    tmp.append(itos[id])
print(''.join(tmp))

▁xxfld▁1▁『おひさま』は、2011年4月4日から同年10月1日まで、▁xxup▁nhkで放送された『連続テレビ小説』第84シリーズの作品。『連続テレビ小説』が『娘と私』放送開始から数えて50周年となるのを記念する作品として位置付けられた。


In [18]:
stoi['h']

660

In [19]:
TextLMDataBunch??

In [20]:
tokenizer = sp['tokenizer']

In [ ]:
tokenizer.special_cases

In [ ]:
tokenizer.tok_func

In [ ]:
tok = tokenizer.process_all(['0\n秋元与助'])

In [ ]:
tok